In [110]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization
import seaborn as sns
import smogn

# 讀取 Excel 表
df1 = pd.read_excel('/Users/ccit0915/Desktop/論文素材/論文data彙整1130413.xlsx', sheet_name='2024-1月')
df2 = pd.read_excel('/Users/ccit0915/Desktop/論文素材/論文data彙整1130413.xlsx', sheet_name='2024-2月')
df3 = pd.read_excel('/Users/ccit0915/Desktop/論文素材/論文data彙整1130413.xlsx', sheet_name='2024-3月')
df4 = pd.read_excel('/Users/ccit0915/Desktop/論文素材/論文data彙整1130413.xlsx', sheet_name='2024-4月')
df5 = pd.read_excel('/Users/ccit0915/Desktop/論文素材/論文data彙整1130413.xlsx', sheet_name='2024-5月')
df6 = pd.read_excel('/Users/ccit0915/Desktop/論文素材/論文data彙整1130413.xlsx', sheet_name='2024-6月')


# 合併 DataFrame
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

# 顯示合併後的 DataFrame
print(df)

                  原料編號      產品編號   數量       下料日期  單片排版數   PCS總數 客戶名稱
0     1BT-30M10PZZ152D  7P05-N72  144 2024-01-02    600   86400  P05
1     1BT-30M10PZZ152D  7P05-N90  384 2024-01-02    880  337920  P05
2     1BT-44N10PZZ182L  7A66-DD7   48 2024-01-02   1280   61440  A66
3     1BT-44N10PZZ182L  7A67-J03   96 2024-01-02   2750  264000  A67
4     1BT-44N10PZZ182L  7A67-J27   48 2024-01-02   1350   64800  A67
...                ...       ...  ...        ...    ...     ...  ...
2785  1BT-43N10PZZ112L  7Q51-642   60 2024-06-30   1120   67200  A66
2786  1BT-43N10PZZ112L  7Q56-452  180 2024-06-30   1680  302400  L08
2787  1BT-43N10PZZ112L  7Q51-452   40 2024-06-30   1680   67200  A66
2788  1BT-62A06P32252M  7Q52-707   96 2024-06-30   3456  331776  P01
2789  1BT-62J04P32252M  7S20-754  192 2024-06-30    600  115200  S20

[2790 rows x 7 columns]


In [111]:
# 找出稀少類別的函數
def find_rare_categories(df, columns):
    rare_categories = {}
    for column in columns:
        counts = df[column].value_counts()
        rare = counts[counts == 1].index.tolist()  # 篩選只出現一次的類別
        if rare:
            rare_categories[column] = rare
    return rare_categories

# 找出稀少類別
columns_to_check = ['原料編號', '客戶名稱']
rare_categories = find_rare_categories(df, columns_to_check)

# 刪除包含稀少類別的行數
for column, rare in rare_categories.items():
    df = df[~df[column].isin(rare)]  # 用 ~ 排除稀少類別的行數

# 顯示刪除後的 DataFrame
print(df)

# 檢查刪除情況
for column, rare in rare_categories.items():
    print(f"刪除的稀少類別 ({column}): {rare}")

                  原料編號      產品編號   數量       下料日期  單片排版數   PCS總數 客戶名稱
0     1BT-30M10PZZ152D  7P05-N72  144 2024-01-02    600   86400  P05
1     1BT-30M10PZZ152D  7P05-N90  384 2024-01-02    880  337920  P05
2     1BT-44N10PZZ182L  7A66-DD7   48 2024-01-02   1280   61440  A66
3     1BT-44N10PZZ182L  7A67-J03   96 2024-01-02   2750  264000  A67
4     1BT-44N10PZZ182L  7A67-J27   48 2024-01-02   1350   64800  A67
...                ...       ...  ...        ...    ...     ...  ...
2785  1BT-43N10PZZ112L  7Q51-642   60 2024-06-30   1120   67200  A66
2786  1BT-43N10PZZ112L  7Q56-452  180 2024-06-30   1680  302400  L08
2787  1BT-43N10PZZ112L  7Q51-452   40 2024-06-30   1680   67200  A66
2788  1BT-62A06P32252M  7Q52-707   96 2024-06-30   3456  331776  P01
2789  1BT-62J04P32252M  7S20-754  192 2024-06-30    600  115200  S20

[2786 rows x 7 columns]
刪除的稀少類別 (原料編號): ['1BT-33J0KP32252D', '1BT-66T05P32212M']
刪除的稀少類別 (客戶名稱): ['P03', 'T80']


In [112]:
df.drop(columns=['單片排版數', '數量'], inplace=True)
print(df.head())

               原料編號      產品編號       下料日期   PCS總數 客戶名稱
0  1BT-30M10PZZ152D  7P05-N72 2024-01-02   86400  P05
1  1BT-30M10PZZ152D  7P05-N90 2024-01-02  337920  P05
2  1BT-44N10PZZ182L  7A66-DD7 2024-01-02   61440  A66
3  1BT-44N10PZZ182L  7A67-J03 2024-01-02  264000  A67
4  1BT-44N10PZZ182L  7A67-J27 2024-01-02   64800  A67


In [113]:
# 對'PCS總數' 進行對數轉換
df['PCS總數'] = df['PCS總數'].apply(lambda x: np.log(x))

print(df.head())

               原料編號      產品編號       下料日期      PCS總數 客戶名稱
0  1BT-30M10PZZ152D  7P05-N72 2024-01-02  11.366743  P05
1  1BT-30M10PZZ152D  7P05-N90 2024-01-02  12.730564  P05
2  1BT-44N10PZZ182L  7A66-DD7 2024-01-02  11.025816  A66
3  1BT-44N10PZZ182L  7A67-J03 2024-01-02  12.483704  A67
4  1BT-44N10PZZ182L  7A67-J27 2024-01-02  11.079061  A67


In [114]:
# 先將 '下料日期' 欄位轉換為日期時間格式
df['下料日期'] = pd.to_datetime(df['下料日期'])

# 提取年、月、日特徵
df['year'] = df['下料日期'].dt.year
df['mon'] = df['下料日期'].dt.month
df['day'] = df['下料日期'].dt.day

In [115]:
# 先根據 'mon' 和 'day' 進行排序
df = df.sort_values(by=['mon', 'day'])

# 根據時間順序來切割數據，例如選擇前 80% 作為訓練集，剩餘 20% 作為測試集
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# 定義訓練集和測試集的特徵和目標變數
X_train = train_data[['產品編號', 'mon', 'day', '客戶名稱', '原料編號']]
y_train = train_data['PCS總數']

X_test = test_data[['產品編號', 'mon', 'day', '客戶名稱', '原料編號']]
y_test = test_data['PCS總數']

In [122]:
import category_encoders as ce

# 創建 X_train 的副本
X_train_copy = X_train.copy()

# 創建目標編碼器
target_encoder_train = ce.TargetEncoder(cols=['產品編號'])
X_train_copy['product_encoded'] = target_encoder_train.fit_transform(X_train_copy['產品編號'], y_train)

# 計算 '客戶名稱' 的頻率
customer_freq_train = X_train_copy['客戶名稱'].value_counts(normalize=True)
X_train_copy['client_encoded'] = X_train_copy['客戶名稱'].map(customer_freq_train)

# 計算 '原料編號' 的頻率
material_freq_train = X_train_copy['原料編號'].value_counts(normalize=True)
X_train_copy['raw_material_encoded'] = X_train_copy['原料編號'].map(material_freq_train)

# 如果需要，將修改後的 X_train_copy 賦回給 X_train
X_train = X_train_copy

In [124]:
# 將 y_train（PCS總數）臨時加入 X_train
X_train['PCS總數'] = y_train

# 計算新特徵
X_train['product_raw_material_interaction_encoded'] = X_train.groupby(['product_encoded', 'raw_material_encoded'])['PCS總數'].transform('mean')
X_train['product_client_interaction_encoded'] = X_train.groupby(['product_encoded', 'client_encoded'])['PCS總數'].transform('mean')
X_train['product_mon'] =  X_train['product_encoded'] * X_train['mon']
X_train['product_day'] =  X_train['product_encoded'] * X_train['day']

# 計算完後，可以選擇再次移除
X_train.drop(columns=['PCS總數'], inplace=True)

In [126]:
X_train.drop(columns=['原料編號'], inplace=True)
X_train.drop(columns=['客戶名稱'], inplace=True)
X_train.drop(columns=['產品編號'], inplace=True)

X_train

,mon,day,product_encoded,client_encoded,raw_material_encoded,product_raw_material_interaction_encoded,product_client_interaction_encoded,product_mon,product_day
0,1,2,11.292342,0.076750,0.026930,11.366743,11.366743,11.292342,22.584683
1,1,2,11.552311,0.076750,0.026930,12.370249,12.370249,11.552311,23.104622
2,1,2,11.214702,0.169659,0.090215,10.891067,10.891067,11.214702,22.429405
3,1,2,11.571846,0.040395,0.090215,12.007166,12.007166,11.571846,23.143693
4,1,2,11.321528,0.040395,0.090215,11.527001,11.527001,11.321528,22.643055
...,...,...,...,...,...,...,...,...,...
2226,5,31,13.717791,0.169659,0.043986,13.742294,13.742294,68.588953,425.251507
2227,5,31,11.545774,0.169659,0.043986,11.942196,11.942196,57.728868,357.918980
2228,5,31,11.490890,0.058797,0.014811,12.086547,12.009489,57.454449,356.217586
2229,6,1,11.282521,0.076750,0.027379,11.290749,11.290749,67.695129,11.282521


In [128]:
# 用訓練集的 target_encoder 來轉換測試集
X_test.loc[:, 'product_encoded'] = target_encoder_train.transform(X_test['產品編號'])

# 用訓練集的頻率來轉換測試集
X_test.loc[:, 'client_encoded'] = X_test['客戶名稱'].map(customer_freq_train)
X_test.loc[:, 'raw_material_encoded'] = X_test['原料編號'].map(material_freq_train)

/var/folders/2y/0mtp5mr56s73ff2f7_z800qr0000gn/T/ipykernel_50506/808990294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:, 'product_encoded'] = target_encoder_train.transform(X_test['產品編號'])
/var/folders/2y/0mtp5mr56s73ff2f7_z800qr0000gn/T/ipykernel_50506/808990294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:, 'client_encoded'] = X_test['客戶名稱'].map(customer_freq_train)


In [130]:
# 將 y_test（PCS總數）臨時加入 X_test
X_test['PCS總數'] = y_test

# 計算新特徵
X_test['product_raw_material_interaction_encoded'] = X_test.groupby(['product_encoded', 'raw_material_encoded'])['PCS總數'].transform('mean')
X_test['product_client_interaction_encoded'] = X_test.groupby(['product_encoded', 'client_encoded'])['PCS總數'].transform('mean')
X_test['product_mon'] =  X_test['product_encoded'] * X_test['mon']
X_test['product_day'] =  X_test['product_encoded'] * X_test['day']

# 計算完後，可以選擇再次移除
X_test.drop(columns=['PCS總數'], inplace=True)

In [132]:
X_test.drop(columns=['原料編號'], inplace=True)
X_test.drop(columns=['客戶名稱'], inplace=True)
X_test.drop(columns=['產品編號'], inplace=True)

X_test

,mon,day,product_encoded,client_encoded,raw_material_encoded,product_raw_material_interaction_encoded,product_client_interaction_encoded,product_mon,product_day
2231,6,1,11.280043,0.076750,0.005386,11.738660,11.391484,67.680260,11.280043
2232,6,1,12.158565,0.083932,0.090215,11.932620,11.932620,72.951391,12.158565
2233,6,1,10.520790,0.144075,0.114004,10.054022,10.054022,63.124738,10.520790
2234,6,1,11.144992,0.144075,0.114004,10.688051,10.688051,66.869955,11.144992
2235,6,1,9.895019,0.132855,0.115350,9.347427,9.347427,59.370113,9.895019
...,...,...,...,...,...,...,...,...,...
2785,6,30,11.124810,0.169659,0.189856,11.069259,11.069259,66.748857,333.744287
2786,6,30,11.697656,0.058797,0.189856,12.088286,12.088286,70.185937,350.929683
2787,6,30,11.801073,0.169659,0.189856,11.704039,11.704039,70.806438,354.032190
2788,6,30,11.331567,0.012567,0.043986,12.437562,12.492493,67.989400,339.947001


In [136]:
y_train

0       11.366743
1       12.730564
2       11.025816
3       12.483704
4       11.079061
          ...    
2226    11.901285
2227    11.613603
2228    11.238910
2229    10.778956
2230    11.472103
Name: PCS總數, Length: 2228, dtype: float64

In [138]:
y_test

2231    12.578684
2232    12.858398
2233    10.227309
2234    10.140297
2235     9.911456
          ...    
2785    11.115429
2786    12.619506
2787    11.115429
2788    12.712215
2789    11.654425
Name: PCS總數, Length: 558, dtype: float64

In [148]:
X_test['raw_material_encoded'].fillna(X_test['raw_material_encoded'].median(), inplace=True)
X_test['product_raw_material_interaction_encoded'].fillna(X_test['product_raw_material_interaction_encoded'].median(), inplace=True)

print(X_test.isnull().sum())  # 再次檢查缺失值

mon                                         0
day                                         0
product_encoded                             0
client_encoded                              0
raw_material_encoded                        0
product_raw_material_interaction_encoded    0
product_client_interaction_encoded          0
product_mon                                 0
product_day                                 0
dtype: int64


/var/folders/2y/0mtp5mr56s73ff2f7_z800qr0000gn/T/ipykernel_50506/128847296.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_test['raw_material_encoded'].fillna(X_test['raw_material_encoded'].median(), inplace=True)
/var/folders/2y/0mtp5mr56s73ff2f7_z800qr0000gn/T/ipykernel_50506/128847296.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate 

In [150]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# 使用 MinMaxScaler 來標準化 X_train 和 X_test
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_test_pred = np.nan_to_num(y_test_pred, nan=0)  # 將 NaN 值替換為 0 或其他合適的值

# LSTM 需要 3D 輸入 [samples, timesteps, features]
# 這裡我們假設每個樣本的時間步長是 1，並且每個樣本有多個特徵
X_train_lstm = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# 構建 LSTM 模型
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(1))  # 輸出一個預測值
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# 訓練模型
model.fit(X_train_lstm, y_train, epochs=20, batch_size=32, validation_data=(X_test_lstm, y_test))

# 計算訓練集的評估指標
y_train_pred = model.predict(X_train_lstm)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# 計算測試集的評估指標
y_test_pred = model.predict(X_test_lstm)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# 輸出結果
print(f"Train MSE: {train_mse}")
print(f"Train RMSE: {train_rmse}")
print(f"Train MAE: {train_mae}")
print(f"Train R²: {train_r2}")

print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R²: {test_r2}")

Epoch 1/20
70/70 [==============================] - 1s 5ms/step - loss: 109.2558 - val_loss: 45.3348
Epoch 2/20
70/70 [==============================] - 0s 1ms/step - loss: 15.2118 - val_loss: 1.4188
Epoch 3/20
70/70 [==============================] - 0s 1ms/step - loss: 1.7944 - val_loss: 1.8197
Epoch 4/20
70/70 [==============================] - 0s 1ms/step - loss: 1.6863 - val_loss: 1.7428
Epoch 5/20
70/70 [==============================] - 0s 1ms/step - loss: 1.6088 - val_loss: 1.7363
Epoch 6/20
70/70 [==============================] - 0s 1ms/step - loss: 1.5325 - val_loss: 1.6341
Epoch 7/20
70/70 [==============================] - 0s 1ms/step - loss: 1.4558 - val_loss: 1.5904
Epoch 8/20
70/70 [==============================] - 0s 1ms/step - loss: 1.3763 - val_loss: 1.5239
Epoch 9/20
70/70 [==============================] - 0s 1ms/step - loss: 1.2918 - val_loss: 1.4386
Epoch 10/20
70/70 [==============================] - 0s 1ms/step - loss: 1.2011 - val_loss: 1.3723
Epoch 11/20
70/